<a href="https://colab.research.google.com/github/aaletov/math-basics-for-ai/blob/master/solutions/msai23_session01_words.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Word vectors**


In the previous exercise we observed that colors that we think of as similar are 'closer' to each other in RGB vector space. Is it possible to create a vector space for all English words that has this same 'closer in space is closer in meaning' property?

The answer is yes! Luckily, you don't need to create those vectors from scratch. Many researchers have made downloadable databases of pre-trained vectors. One such project is [Stanford's Global Vectors for Word Representation (GloVe)](https://nlp.stanford.edu/projects/glove/).

These $300$-dimensional vectors are included with $\texttt{spaCy}$, and they're the vectors we'll be using in this exercise.

![cosine similarity: picture](https://d33wubrfki0l68.cloudfront.net/d2742976a92aa4d6c39f19c747ec5f56ed1cec30/3803f/images/guide-to-word-vectors-with-gensim-and-keras_files/word2vec-king-queen-vectors.png)

In [1]:
# The following will download the language model.
# Resart the runtime (Runtime -> Restart runtime) after running this cell
# (and don't run it for the second time).
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


Let's load the model now:

In [1]:
import spacy

nlp = spacy.load('en_core_web_lg')

## **Word vectors: the first glance**

You can see the vector of any word in $\texttt{spaCy}$' s vocabulary using the $\texttt{vector}$ attribute:

In [2]:
# A 300-dimensional vector
len(nlp('dog').vector)

300

In [3]:
nlp('dog').vector

array([ 1.2330e+00,  4.2963e+00, -7.9738e+00, -1.0121e+01,  1.8207e+00,
        1.4098e+00, -4.5180e+00, -5.2261e+00, -2.9157e-01,  9.5234e-01,
        6.9880e+00,  5.0637e+00, -5.5726e-03,  3.3395e+00,  6.4596e+00,
       -6.3742e+00,  3.9045e-02, -3.9855e+00,  1.2085e+00, -1.3186e+00,
       -4.8886e+00,  3.7066e+00, -2.8281e+00, -3.5447e+00,  7.6888e-01,
        1.5016e+00, -4.3632e+00,  8.6480e+00, -5.9286e+00, -1.3055e+00,
        8.3870e-01,  9.0137e-01, -1.7843e+00, -1.0148e+00,  2.7300e+00,
       -6.9039e+00,  8.0413e-01,  7.4880e+00,  6.1078e+00, -4.2130e+00,
       -1.5384e-01, -5.4995e+00,  1.0896e+01,  3.9278e+00, -1.3601e-01,
        7.7732e-02,  3.2218e+00, -5.8777e+00,  6.1359e-01, -2.4287e+00,
        6.2820e+00,  1.3461e+01,  4.3236e+00,  2.4266e+00, -2.6512e+00,
        1.1577e+00,  5.0848e+00, -1.7058e+00,  3.3824e+00,  3.2850e+00,
        1.0969e+00, -8.3711e+00, -1.5554e+00,  2.0296e+00, -2.6796e+00,
       -6.9195e+00, -2.3386e+00, -1.9916e+00, -3.0450e+00,  2.48

## **Cosine similarity**

**Cosine similarity** is a common way of assessing similarity between words in NLP. It is essentially defined as the cosine of the angle between the vectors representing the words of interest.

Recall that the angle $\phi$ between two non-zero vectors $u$ and $v$ can be computed as follows:

$cos(\phi) = \frac{(u,v)}{||u||\cdot||v||}$

![](https://miro.medium.com/max/1394/1*_Bf9goaALQrS_0XkBozEiQ.png)



Define a function computing cosine similarity between two vectors.

In [4]:
import numpy as np

def cosine(v1, v2):
  return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

Test your function by computing similarities of some random pairs of words, e.g. $dog$ and $puppy$ vs. $dog$ and $kitten$.

In [7]:
print(cosine(nlp("dog").vector, nlp("dog").vector))
print(cosine(nlp("dog").vector, nlp("puppy").vector))
print(cosine(nlp("dog").vector, nlp("kitten").vector))
print(cosine(nlp("dog").vector, nlp("kindergarden").vector))

1.0000001
0.81076676
0.6515032
0.24433023


## **Loading the text**

Let's load the full text of *Alice in Wonderland*. It will serve us as a corpus of English words.

In [8]:
import requests

# Alice in Wonderland
response = requests.get('https://www.gutenberg.org/files/11/11-0.txt')

# If you prefer Dracula, load this instead:
#response = requests.get('https://www.gutenberg.org/cache/epub/345/pg345.txt')

# Extracting separate words from the text
doc = nlp(response.text)
tokens = list(set([w.text for w in doc if w.is_alpha]))

Check out the content of $\texttt{tokens}$ now.

In [9]:
tokens

['atom',
 'flame',
 'consider',
 'zigzag',
 'feelings',
 'reduced',
 'applause',
 'thousand',
 'completely',
 'In',
 'quarrel',
 'introduced',
 'every',
 'books',
 'clearer',
 'Little',
 'but',
 'rats',
 'plan',
 'occasional',
 'Down',
 'if',
 'pairs',
 'soldier',
 'thoroughly',
 'three',
 'welcome',
 'accidentally',
 'Twinkle',
 'walking',
 'banks',
 'fell',
 'drink',
 'muttered',
 'consented',
 'into',
 'Knave',
 'promise',
 'rich',
 'stop',
 'finger',
 'between',
 'cheered',
 'picking',
 'rule',
 'round',
 'unwillingly',
 'passed',
 'flavour',
 'ringlets',
 'sorrow',
 'secret',
 'indignantly',
 'growled',
 'ferrets',
 'nodded',
 'empty',
 'our',
 'Then',
 'puppy',
 'time',
 'taking',
 'delightful',
 'delay',
 'Lastly',
 'tones',
 'experiment',
 'beautiful',
 'Bill',
 'roses',
 'wrote',
 'out',
 'crown',
 'languid',
 'clasped',
 'form',
 'hastily',
 'natured',
 'doubt',
 'March',
 'II',
 'impatient',
 'cares',
 'or',
 'filled',
 'tumbled',
 'leap',
 'shutting',
 'invited',
 'sell',
 

Define a function that takes a word and lists the $n$ most similar words in our corpus.

In [51]:
def spacy_closest(tokens, new_vec, n=10):
  closest = sorted(tokens, key=lambda token: cosine(nlp(token).vector, new_vec), reverse=True)
  return closest[:n]

Try to find words similar to some random words, e.g. $good$.

In [52]:
spacy_closest(tokens, nlp('good').vector)

<ipython-input-4-958c4f6c629b>:4: RuntimeWarning: invalid value encountered in scalar divide
  return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))


['wonderful',
 'pleasant',
 'happy',
 'best',
 'poor',
 'always',
 'like',
 'comfortable',
 'unpleasant',
 'fortunately']

You can also get creative and search for combinations of words. For example, what is similar to $king - man + woman$?

In [53]:
# Your code here
spacy_closest(tokens, nlp("king").vector - nlp("man").vector + nlp("woman").vector, n=5)

<ipython-input-4-958c4f6c629b>:4: RuntimeWarning: invalid value encountered in scalar divide
  return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))


['king', 'crown', 'consented', 'authority', 'ordering']

## **Sentence vectors**

We can also construct a vector representation for the whole sentence. For example, we can define it as an *average* of the   vectors representing the words in it.

Let's take a random sentence *My favorite food is strawberry ice cream* and construct its vector representation.

In [67]:
sent = nlp('My favorite food is strawberry ice cream.')

# Your code here
whole = 'My favorite food is strawberry ice cream.'
def sentence_vector(sentence):
    calculator = lambda token: nlp(token).vector
    return sum(map(calculator, sentence.split())) / len(sentence.split())

sentv = sentence_vector(whole)

Let's also extract sentences (as opposed to individual words) from our corpus:

In [68]:
sents = list(doc.sents)

Define a function that takes a random sentence and lists $n$ most similar sentences from our corpus.

In [76]:
def spacy_closest_sent(sentences, input_vec, n=10):
  print(type(sentences[0]))
  closest = sorted(sentences, key=lambda sent: cosine(nlp(sent).vector, input_vec))
  return sorted[:n]

Let's try it out!

In [77]:
for s in spacy_closest_sent(sents, sentv, n=10):
  print(s)
  print('\n---')

<class 'spacy.tokens.span.Span'>


ValueError: [E1041] Expected a string, Doc, or bytes as input, but got: <class 'spacy.tokens.span.Span'>

## **References**

This notebook is inspired by a [tutorial by Allison Parrish](https://gist.github.com/aparrish/2f562e3737544cf29aaf1af30362f469).